In [1]:
from api import inoreader_api, inoreader_scraping
import json
import pandas as pd
import re
import random
import pickle


def rewrite(x):
    return x

# Scrape feeds starting from a concrete moment and store them in a file.

In [2]:
start_time = 1508112001
feed = 'user/-/label/arXiv'
file_name = 'articles_raw.json'

number_scraped = inoreader_scraping.scrape(feed, start_time, file_name)
print(number_scraped)


Requesting: stream/contents/user/-/label/arXiv
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=t_CG8hNj8aj6
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=8xuq3sUctGGH
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=rZqJsXROlADA
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=DqPQF0yrjOrx
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=po4gGSY1J9uX
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=QDDeL_9IdUjP
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=pUOKPZnUUiaR
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=iEqoZOQcN_C1
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=MCkRKbSJjLYH
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=_5jC8ZR9_uhI
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=9Pjwy7JaU6Ol
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=ADKZJQsu26Wo
200 OK

Requesting: stream/contents/user/-/label/arXiv?c=YHyXWjta9pxk
200 OK

Requesting: stream/contents/user/-

# Load the feeds from disk.

In [3]:
f = open(file_name, 'r')
json_string = f.read()
f.close

json_object = json.loads(json_string)
all_articles = json_object['all_articles']

In [4]:
def get_abstract_string(abs_dict):
    abs_string = abs_dict['content']
    abs_string = re.search('<p>(.*)</p>', abs_string, flags=re.DOTALL)
    return abs_string.group(1)

In [5]:
def get_canonical_string(canonical_list):
    url_dict = canonical_list[0]
    url = url_dict['href']
    return url

# Create preprocessed and cleaned DataFrame.

In [6]:
all_articles_pd = pd.DataFrame(all_articles)
all_articles_pd = all_articles_pd[['canonical', 'author', 'categories', 'published', 'title', 'summary']]

for i in range(len(all_articles_pd)):
    all_articles_pd['summary'][i] = get_abstract_string(all_articles_pd['summary'][i])
    all_articles_pd['canonical'][i] = get_canonical_string(all_articles_pd['canonical'][i])
    
all_articles_pd['read'] = False
all_articles_pd['liked'] = False

for i in range(len(all_articles_pd)):
    if 'user/1005689817/state/com.google/read' in all_articles_pd['categories'][i]:
        all_articles_pd['read'][i] = True
    if 'user/1005689817/state/com.google/like' in all_articles_pd['categories'][i]:
        all_articles_pd['liked'][i] = True

del all_articles_pd['categories']

/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyW

In [7]:
#all_articles_pd

In [8]:
tagged_articles_pd = all_articles_pd[all_articles_pd['read'] == True]

with open('old_articles_proto2.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    old_tagged_articles_pd = pickle.load(f)
    
tagged_articles_pd = pd.concat([tagged_articles_pd, old_tagged_articles_pd])

In [9]:
tagged_articles_unique_pd = tagged_articles_pd.sort_values(by=['canonical', 'liked'], ascending=False).reset_index(drop=True)
to_drop = tagged_articles_unique_pd.duplicated(subset = ['canonical'], keep='first')
to_drop = list(to_drop[to_drop == False].index.values)

tagged_articles_unique_pd = tagged_articles_unique_pd[tagged_articles_unique_pd.index.isin(to_drop)]

In [10]:
print(len(tagged_articles_pd))
print(len(tagged_articles_unique_pd))

2716
2501


In [11]:
X_ = list(tagged_articles_unique_pd['author']+' '+tagged_articles_unique_pd['title']+' '+tagged_articles_unique_pd['summary'])
y_ = list(tagged_articles_unique_pd['liked'])

In [12]:
index_shuffled = [i for i in range(len(X_))]
random.shuffle(index_shuffled)
X = []
y = []
for i in index_shuffled:
    X.append(X_[i])
    y.append(y_[i])

In [13]:
with open('stopwords/english') as f:
    stopwords = [line.strip() for line in f]

In [14]:


def text_to_word_sequence(text,
                          filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                          lower=True, split=" "):
    if lower: text = text.lower()
    if type(text) == str:
        translate_table = {ord(c): ord(t) for c,t in zip(filters, split*len(filters)) }
    else:
        translate_table = maketrans(filters, split * len(filters))
    text = text.translate(translate_table)
    seq = text.split(split)
    return [i for i in seq if i]

print(len(X[4]))

for i in range(len(X)):
    seq = text_to_word_sequence(X[i])
    clean_seq = [word for word in seq if word not in stopwords]
    X[i] = ' '.join(clean_seq)
    
print(len(X[4]))

1161
950


# Saving or loading the data and LE and tokenizer

In [15]:
import pickle

data_dict = {
    'X' : X,
    'y' : y
}
pickle.dump(data_dict,open("data.pickle", "wb" ) )


limit = int(0.75*len(X))
print(limit)

1875


In [16]:
import pickle
data_dict = pickle.load(open( "data.pickle", "rb" ) )
X = data_dict['X']
y = data_dict['y']
limit = int(0.75*len(X))
print(limit)

1875


In [17]:
X_ = X
y_ = y
index_shuffled = [i for i in range(len(X_))]
random.shuffle(index_shuffled)
X = []
y = []
for i in index_shuffled:
    X.append(X_[i])
    y.append(y_[i])

# TFIDF -> NB

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit


model_nb_tfidf = Pipeline([
    ("tfidf_vectorizer", TfidfVectorizer()),\
    ("nb", BernoulliNB())
])


X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

model_nb_tfidf.fit(X_train, y_train)

y_predicted = model_nb_tfidf.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
acc, f1

/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(0.84664536741214058, 0.9153791382943729)

In [19]:
y_predicted_proba = model_nb_tfidf.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.15)
y_predicted = (y_predicted_proba[:,1] > 0.10)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)/len(y_predicted)*100.0,"% - ", int(sum(y_predicted)/len(y_predicted)*50.0))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

0.846645367412 0.915379138294 1 626
0.159744408946 % -  0


Rejected 100.00% of wrong ones
Accepted 1.03% of good ones


array([[  1.00000000e+00,   8.30570688e-27],
       [  1.00000000e+00,   9.85238929e-31],
       [  1.00000000e+00,   1.72855324e-18],
       [  1.00000000e+00,   3.32406901e-20],
       [  1.00000000e+00,   6.48447509e-29]])

# CountVect -> NB

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit


model_nb = Pipeline([
    #('vect', CountVectorizer(ngram_range=(1,3),stop_words='english')),\
    ("count_vectorizer", CountVectorizer(analyzer=rewrite)),\
    ("nb", BernoulliNB())
])

X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

model_nb.fit(X_train, y_train)

y_predicted = model_nb.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
acc, f1

/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.84504792332268375, 0.91601731601731617)

In [21]:
y_predicted_proba = model_nb.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.1296)
y_predicted = (y_predicted_proba[:,1] > 0.1325)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

0.533546325879 0.46994372247 305 626
48.7220447284 % -  24


Rejected 52.74% of wrong ones
Accepted 56.70% of good ones


array([[ 0.83709096,  0.16290904],
       [ 0.89966041,  0.10033959],
       [ 0.86289978,  0.13710022],
       [ 0.82382712,  0.17617288],
       [ 0.90048481,  0.09951519]])

# TFIDF -> SVC

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit

model_svc_tfidf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,3), min_df=1, max_df=1.0,stop_words='english')),
    #('vect', CountVectorizer(ngram_range=(3,6),analyzer='char_wb', min_df=10, max_df=0.95,stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ("svc", SVC(kernel='linear', probability=True, class_weight = 'balanced'))
    #("svc", SVC(kernel='linear', probability=True))
])

X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

model_svc_tfidf.fit(X_train, y_train)

y_predicted = model_svc_tfidf.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
acc, f1

(0.85942492012779548, 0.90380838196886393)

In [23]:
y_predicted_proba = model_svc_tfidf.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.103)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

#(0.62077922077922076, 0.55417162908266104, 162, 385)
#(42.077922077922075, '% - ', 21)
#('Rejected good ones: ', 15, '/', 46)
#('Accepted wrong ones:', 131, '/', 339)
#('Rejected wrong ones: ', 208, '/', 339)
#('Accepted good ones:', 31, '/', 46)

0.543130990415 0.483110192189 331 626
52.875399361 % -  26


Rejected 50.85% of wrong ones
Accepted 73.20% of good ones


array([[ 0.86538007,  0.13461993],
       [ 0.95301607,  0.04698393],
       [ 0.54068916,  0.45931084],
       [ 0.48593804,  0.51406196],
       [ 0.92803788,  0.07196212]])

In [24]:
(len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/len(y_test)-sum(y_test))


-54.028753993610223

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit

model_svc_tfidf = Pipeline([
    #('vect', CountVectorizer(ngram_range=(1,3), min_df=1, max_df=1.0,stop_words='english')),
    ('vect', CountVectorizer(ngram_range=(3,6),analyzer='char_wb', min_df=10, max_df=0.95,stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ("svc", SVC(kernel='linear', probability=True, class_weight = 'balanced'))
    #("svc", SVC(kernel='linear', probability=True))
])

X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

model_svc_tfidf.fit(X_train, y_train)

y_predicted = model_svc_tfidf.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
acc, f1

(0.74281150159744413, 0.72492789072766561)

In [26]:
y_predicted_proba = model_svc_tfidf.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.111)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

#(0.62077922077922076, 0.55417162908266104, 162, 385)
#(42.077922077922075, '% - ', 21)
#('Rejected good ones: ', 15, '/', 46)
#('Accepted wrong ones:', 131, '/', 339)
#('Rejected wrong ones: ', 208, '/', 339)
#('Accepted good ones:', 31, '/', 46)

0.511182108626 0.450496232577 355 626
56.7092651757 % -  28


Rejected 46.69% of wrong ones
Accepted 75.26% of good ones


array([[ 0.92043355,  0.07956645],
       [ 0.91330185,  0.08669815],
       [ 0.73624124,  0.26375876],
       [ 0.6567993 ,  0.3432007 ],
       [ 0.86044217,  0.13955783]])

# Count -> SVC

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit

model_svc = Pipeline([
    #('vect', CountVectorizer()),
    ('vect', CountVectorizer(ngram_range=(1,3), min_df=1, max_df=1.0,stop_words='english')),
    #('vect', CountVectorizer(ngram_range=(3,6),analyzer='char_wb', min_df=10, max_df=0.95,stop_words='english')),
    #('tfidf', TfidfTransformer()),
    ("svc", SVC(kernel='linear', probability=True, class_weight = 'balanced'))
    #("svc", SVC(kernel='linear', probability=True))
])

X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

model_svc.fit(X_train, y_train)

y_predicted = model_svc.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
acc, f1

(0.86102236421725242, 0.91004330704814829)

In [28]:
y_predicted_proba = model_svc.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.11)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

#(0.62077922077922076, 0.55417162908266104, 162, 385)
#(42.077922077922075, '% - ', 21)
#('Rejected good ones: ', 15, '/', 46)
#('Accepted wrong ones:', 131, '/', 339)
#('Rejected wrong ones: ', 208, '/', 339)
#('Accepted good ones:', 31, '/', 46)

0.506389776358 0.443629863022 346 626
55.2715654952 % -  27


Rejected 47.26% of wrong ones
Accepted 69.07% of good ones


array([[ 0.87842511,  0.12157489],
       [ 0.93458029,  0.06541971],
       [ 0.66181316,  0.33818684],
       [ 0.75052072,  0.24947928],
       [ 0.88606537,  0.11393463]])

# Extree - pretrained

In [29]:
test = ['raz']
'_'.join(test)

'raz'

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit
from gensim.models.word2vec import Word2Vec

import numpy as np

import models.embedding_matrix as embedding
#reload(embedding)

DATASETS_DIR = '../ml-research/datasets/'
WIKI_DIR = DATASETS_DIR+'wiki.pl/'
embeddings_file = WIKI_DIR+'wiki.pl.vec'

print('Indexing word vectors.')

word2vec_pretrained = {}
f = open(embeddings_file, encoding='utf-8')

for line in f:
    values = line.split()
    word = '_'.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    word2vec_pretrained[word] = coefs
f.close()

# train word2vec on all the texts - both training and test set
# we're not using test labels, just texts so this is fine
model = Word2Vec(X, size=100, window=5, min_count=5, workers=2)
w2v_custom = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

Indexing word vectors.


In [31]:
from collections import defaultdict


class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(word2vec.items())))
        
    
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
# and a tf-idf version of the same

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(word2vec.items())))
        
    def max_idf_func():
        return self.max_idf
        
    def fit(self, X, y):
        #tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf = TfidfVectorizer(analyzer=rewrite)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        self.max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            #lambda: max_idf, 
            self.max_idf_func, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
        return self
    
    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [32]:
# Extra Trees classifier is almost universally great, let's stack it with our embeddings

etree_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(word2vec_pretrained)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=50))])
etree_w2v_tfidf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(word2vec_pretrained)), 
                        ("svc", ExtraTreesClassifier(n_estimators=50))])
svc_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(word2vec_pretrained)), 
                        ("extra trees", SVC(kernel="linear", probability=True))])
svc_w2v_tfidf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(word2vec_pretrained)), 
                        ("svc", SVC(kernel="linear", probability=True))])

In [33]:
import numpy as np
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

etree_w2v.fit(X_train, y_train)

y_predicted = etree_w2v.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = etree_w2v.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.14)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

0.546325878594 0.483668515517 275 626
43.9297124601 % -  21


Rejected 56.33% of wrong ones
Accepted 45.36% of good ones


array([[ 0.92,  0.08],
       [ 0.76,  0.24],
       [ 0.78,  0.22],
       [ 0.62,  0.38],
       [ 0.74,  0.26]])

In [34]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

etree_w2v_tfidf.fit(X_train, y_train)

y_predicted = etree_w2v_tfidf.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = etree_w2v_tfidf.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.137)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

0.492012779553 0.426956064207 343 626
54.7923322684 % -  27


Rejected 46.69% of wrong ones
Accepted 62.89% of good ones


array([[ 0.96,  0.04],
       [ 0.78,  0.22],
       [ 0.76,  0.24],
       [ 0.74,  0.26],
       [ 0.86,  0.14]])

In [35]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

svc_w2v.fit(X_train, y_train)

y_predicted = svc_w2v.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = svc_w2v.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.135)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.26517571885 0.260382408155 523 626
83.5463258786 % -  41


Rejected 16.26% of wrong ones
Accepted 82.47% of good ones


array([[ 0.85510646,  0.14489354],
       [ 0.85092879,  0.14907121],
       [ 0.86251459,  0.13748541],
       [ 0.85062153,  0.14937847],
       [ 0.8747115 ,  0.1252885 ]])

In [36]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

svc_w2v_tfidf.fit(X_train, y_train)

y_predicted = svc_w2v_tfidf.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = svc_w2v_tfidf.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.145)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.434504792332 0.37185657906 387 626
61.821086262 % -  30


Rejected 39.13% of wrong ones
Accepted 67.01% of good ones


array([[ 0.86551962,  0.13448038],
       [ 0.86960114,  0.13039886],
       [ 0.85513103,  0.14486897],
       [ 0.84606409,  0.15393591],
       [ 0.89137907,  0.10862093]])

# Extree - Custom embeddings

In [37]:
# Extra Trees classifier is almost universally great, let's stack it with our embeddings

etree_w2v_custom = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(w2v_custom)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=50))])
etree_w2v_tfidf_custom = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v_custom)), 
                        ("svc", ExtraTreesClassifier(n_estimators=50))])
svc_w2v_custom = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(w2v_custom)), 
                        ("extra trees", SVC(kernel="linear", probability=True))])
svc_w2v_tfidf_custom = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v_custom)), 
                        ("svc", SVC(kernel="linear", probability=True))])

In [38]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

etree_w2v_custom.fit(X_train, y_train)

y_predicted = etree_w2v_custom.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = etree_w2v_custom.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.14)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

0.576677316294 0.518277958272 270 626
43.1309904153 % -  21


Rejected 58.60% of wrong ones
Accepted 52.58% of good ones


array([[ 0.98,  0.02],
       [ 0.8 ,  0.2 ],
       [ 0.92,  0.08],
       [ 0.62,  0.38],
       [ 0.8 ,  0.2 ]])

In [39]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

etree_w2v_tfidf_custom.fit(X_train, y_train)

y_predicted = etree_w2v_tfidf_custom.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = etree_w2v_tfidf_custom.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.137)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

0.477635782748 0.410896327504 344 626
54.9520766773 % -  27


Rejected 45.75% of wrong ones
Accepted 58.76% of good ones


array([[ 0.98,  0.02],
       [ 0.92,  0.08],
       [ 0.86,  0.14],
       [ 0.68,  0.32],
       [ 0.78,  0.22]])

In [40]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

svc_w2v_custom.fit(X_train, y_train)

y_predicted = svc_w2v_custom.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = svc_w2v_custom.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.135)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.311501597444 0.284468217531 490 626
78.2747603834 % -  39


Rejected 22.12% of wrong ones
Accepted 80.41% of good ones


array([[ 0.83214848,  0.16785152],
       [ 0.84460395,  0.15539605],
       [ 0.83287225,  0.16712775],
       [ 0.86248893,  0.13751107],
       [ 0.86622438,  0.13377562]])

In [41]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

svc_w2v_tfidf_custom.fit(X_train, y_train)

y_predicted = svc_w2v_tfidf_custom.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = svc_w2v_tfidf_custom.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.145)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.519169329073 0.455881625663 330 626
52.7156549521 % -  26


Rejected 49.53% of wrong ones
Accepted 64.95% of good ones


array([[ 0.87600593,  0.12399407],
       [ 0.86825217,  0.13174783],
       [ 0.74844131,  0.25155869],
       [ 0.80909423,  0.19090577],
       [ 0.92030731,  0.07969269]])

# Logistic regression and SGD - both on embeddings

In [42]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

logistic_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(word2vec_pretrained)), 
                        ("logistic", LogisticRegression())])
logistic_w2v_tfidf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(word2vec_pretrained)), 
                        ("logistic", LogisticRegression())])

In [43]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

logistic_w2v.fit(X_train, y_train)

y_predicted = logistic_w2v.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = logistic_w2v.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.14)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.58785942492 0.530933297034 281 626
44.8881789137 % -  22


Rejected 58.22% of wrong ones
Accepted 61.86% of good ones


array([[ 0.91300475,  0.08699525],
       [ 0.876287  ,  0.123713  ],
       [ 0.81748738,  0.18251262],
       [ 0.8179039 ,  0.1820961 ],
       [ 0.93249895,  0.06750105]])

In [44]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

logistic_w2v_tfidf.fit(X_train, y_train)

y_predicted = logistic_w2v_tfidf.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = logistic_w2v_tfidf.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.14)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.586261980831 0.529121925201 282 626
45.0479233227 % -  22


Rejected 58.03% of wrong ones
Accepted 61.86% of good ones


array([[ 0.91301352,  0.08698648],
       [ 0.87565938,  0.12434062],
       [ 0.81792818,  0.18207182],
       [ 0.8166771 ,  0.1833229 ],
       [ 0.93235949,  0.06764051]])

In [45]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

sgd_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(word2vec_pretrained)), 
                        ("logistic", SGDClassifier(loss = 'log'))])
sgd_w2v_tfidf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(word2vec_pretrained)), 
                        ("logistic", SGDClassifier(loss = 'log'))])

In [46]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

sgd_w2v.fit(X_train, y_train)

y_predicted = sgd_w2v.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = sgd_w2v.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.14)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

0.845047923323 0.890780343528 20 626
3.19488817891 % -  1


Rejected 98.11% of wrong ones
Accepted 10.31% of good ones


array([[  9.99925987e-01,   7.40129438e-05],
       [  9.99048409e-01,   9.51590837e-04],
       [  9.64826950e-01,   3.51730502e-02],
       [  9.74513822e-01,   2.54861779e-02],
       [  9.99986232e-01,   1.37679391e-05]])

In [47]:
X_train = X[:limit]
X_test = X[limit:]

y_train = y[:limit]
y_test = y[limit:]

sgd_w2v_tfidf.fit(X_train, y_train)

y_predicted = sgd_w2v_tfidf.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = sgd_w2v_tfidf.predict_proba(X_test)
#y_predicted = (y_predicted_proba[:,1] > 0.12)
y_predicted = (y_predicted_proba[:,1] > 0.14)

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))

y_predicted_proba[0:5]

/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.846645367412 0.912465388711 3 626
0.479233226837 % -  0


Rejected 99.81% of wrong ones
Accepted 2.06% of good ones


array([[  9.99981792e-01,   1.82079612e-05],
       [  9.99847635e-01,   1.52365436e-04],
       [  9.95931515e-01,   4.06848476e-03],
       [  9.97686029e-01,   2.31397114e-03],
       [  9.99999189e-01,   8.10905966e-07]])

# Saving/loading models

In [48]:
import time
def datetime_string():
    localtime = time.localtime(time.time())
    datetime_str = str(localtime.tm_year)
    if(localtime.tm_mon < 10):
        datetime_str += '0'
    datetime_str += str(localtime.tm_mon)
    if(localtime.tm_mday < 10):
        datetime_str += '0'
    datetime_str += str(localtime.tm_mday) + '_'
    if(localtime.tm_hour < 10):
        datetime_str += '0'
    datetime_str += str(localtime.tm_hour)
    if(localtime.tm_min < 10):
        datetime_str += '0'
    datetime_str += str(localtime.tm_min) + '_'
    
    return datetime_str

In [49]:
import pickle

contents_dict = {
    'model_svc_sklearn' : model_svc,
    'model_svc_tfidf_sklearn' : model_svc_tfidf,
    
    'model_nb_sklearn' : model_nb,
    'model_nb_tfidf_sklearn' : model_nb_tfidf,
    
    'etree_w2v_sklearn' : etree_w2v,
    'etree_w2v_tfidf_sklearn' : etree_w2v_tfidf,
    
    'svc_w2v_sklearn' : svc_w2v,
    'svc_w2v_tfidf_sklearn' : svc_w2v_tfidf,
    
    'etree_w2v_custom_sklearn' : etree_w2v_custom,
    'etree_w2v_tfidf_custom_sklearn' : etree_w2v_tfidf_custom,
    
    'svc_w2v_custom_sklearn' : svc_w2v_custom,
    'svc_w2v_tfidf_custom_sklearn' : svc_w2v_tfidf_custom,
    
    'logistic_w2v_sklearn' : logistic_w2v,
    'logistic_w2v_tfidf_sklearn' : logistic_w2v_tfidf,
    
    'sgd_w2v_sklearn' : sgd_w2v,
    'sgd_w2v_tfidf_sklearn' : sgd_w2v_tfidf
}

dat_str = datetime_string()
with open("models/"+dat_str+"ensemble_models.pickle", 'wb') as f:
    pickle.dump(contents_dict, f, pickle.HIGHEST_PROTOCOL)

In [50]:
with open('models/'+dat_str+'ensemble_models.pickle', 'rb') as f:
    contents_dict = pickle.load(f)

model_svc = contents_dict['model_svc_sklearn']
model_svc_tfidf = contents_dict['model_svc_tfidf_sklearn']

model_nb = contents_dict['model_nb_sklearn']
model_nb_tfidf = contents_dict['model_nb_tfidf_sklearn']

etree_w2v = contents_dict['etree_w2v_sklearn']
etree_w2v_tfidf = contents_dict['etree_w2v_tfidf_sklearn']

svc_w2v = contents_dict['svc_w2v_sklearn']
svc_w2v_tfidf = contents_dict['svc_w2v_tfidf_sklearn']

etree_w2v_custom = contents_dict['etree_w2v_custom_sklearn']
etree_w2v_tfidf_custom = contents_dict['etree_w2v_tfidf_custom_sklearn']

svc_w2v_custom = contents_dict['svc_w2v_custom_sklearn']
svc_w2v_tfidf_custom = contents_dict['svc_w2v_tfidf_custom_sklearn']

logistic_w2v = contents_dict['logistic_w2v_sklearn']
logistic_w2v_tfidf = contents_dict['logistic_w2v_tfidf_sklearn']

sgd_w2v = contents_dict['sgd_w2v_sklearn']
sgd_w2v_tfidf = contents_dict['sgd_w2v_tfidf_sklearn']

#hyperparameters_cnn = contents_dict['hyperparameters_cnn']
#hyperparameters_lstm = contents_dict['hyperparameters_lstm']

#label_encoder = contents_dict['label_encoder']
#tokenize = contents_dict['tokenizer']

# Building an ensembler - Without data leakage

In [51]:
XE = X[limit:]
yE = y[limit:]

In [52]:
import numpy as np
XE_intermediate = np.column_stack((
        model_svc.predict_proba(XE)[:,1],
        model_svc_tfidf.predict_proba(XE)[:,1],
    
        model_nb.predict_proba(XE)[:,1],
        model_nb_tfidf.predict_proba(XE)[:,1],

        etree_w2v.predict_proba(XE)[:,1],
        etree_w2v_tfidf.predict_proba(XE)[:,1],

        svc_w2v.predict_proba(XE)[:,1],
        svc_w2v_tfidf.predict_proba(XE)[:,1],

        etree_w2v_custom.predict_proba(XE)[:,1],
        etree_w2v_tfidf_custom.predict_proba(XE)[:,1],

        svc_w2v_custom.predict_proba(XE)[:,1],
        svc_w2v_tfidf_custom.predict_proba(XE)[:,1],

        logistic_w2v.predict_proba(XE)[:,1],
        logistic_w2v_tfidf.predict_proba(XE)[:,1],

        sgd_w2v.predict_proba(XE)[:,1],
        sgd_w2v_tfidf.predict_proba(XE)[:,1]
    ))
XE_intermediate

array([[  1.21574889e-01,   7.95664459e-02,   1.62909041e-01, ...,
          8.69864809e-02,   7.40129438e-05,   1.82079612e-05],
       [  6.54197136e-02,   8.66981483e-02,   1.00339593e-01, ...,
          1.24340617e-01,   9.51590837e-04,   1.52365436e-04],
       [  3.38186839e-01,   2.63758763e-01,   1.37100216e-01, ...,
          1.82071820e-01,   3.51730502e-02,   4.06848476e-03],
       ..., 
       [  1.93076372e-01,   1.20739214e-01,   1.16016954e-01, ...,
          1.40714256e-01,   3.81290079e-03,   4.55353799e-04],
       [  1.03825824e-01,   8.29335477e-02,   1.07106650e-01, ...,
          8.67612935e-02,   8.51416976e-05,   1.59641229e-05],
       [  1.01555166e-01,   1.58823533e-01,   1.40582320e-01, ...,
          1.33796650e-01,   2.25872637e-03,   3.55119537e-04]])

In [53]:
limitE = int(0.8*len(XE_intermediate))
print(limitE)

X_train = XE_intermediate[:limitE]
X_test = XE_intermediate[limitE:]

y_train = yE[:limitE]
y_test = yE[limitE:]

500


In [54]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

logi = LogisticRegression()
logi.fit(X_train, y_train)

y_predicted = logi.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = logi.predict_proba(X_test)
y_predicted = (y_predicted_proba[:,1] > np.median(y_predicted_proba[:,1]))

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))
print("Median: %.3f" % np.median(y_predicted_proba[:,1]))

y_predicted_proba[0:5]

0.563492063492 0.485714285714 63 126
50.0 % -  25


Rejected 53.57% of wrong ones
Accepted 78.57% of good ones
Median: 0.149


array([[ 0.87215598,  0.12784402],
       [ 0.86738353,  0.13261647],
       [ 0.8502556 ,  0.1497444 ],
       [ 0.86755385,  0.13244615],
       [ 0.7963666 ,  0.2036334 ]])

# Best ratings:
Rejected 53.94% of wrong ones

Accepted 77.05% of good ones

Median: 0.043

In [ ]:
import numpy as np
max_rej = 20.0
max_acc = 20.0

XE_intermediate_full = np.column_stack((
        model_svc.predict_proba(XE)[:,1],
        model_svc_tfidf.predict_proba(XE)[:,1],

        model_nb.predict_proba(XE)[:,1],
        model_nb_tfidf.predict_proba(XE)[:,1], #----------

        etree_w2v.predict_proba(XE)[:,1],
        etree_w2v_tfidf.predict_proba(XE)[:,1],

        svc_w2v.predict_proba(XE)[:,1],
        svc_w2v_tfidf.predict_proba(XE)[:,1], #----------

        etree_w2v_custom.predict_proba(XE)[:,1],
        etree_w2v_tfidf_custom.predict_proba(XE)[:,1],

        svc_w2v_custom.predict_proba(XE)[:,1],
        svc_w2v_tfidf_custom.predict_proba(XE)[:,1], #----------

        logistic_w2v.predict_proba(XE)[:,1],
        logistic_w2v_tfidf.predict_proba(XE)[:,1],

        sgd_w2v.predict_proba(XE)[:,1],
        sgd_w2v_tfidf.predict_proba(XE)[:,1]
    ))
print("Done!")

Done!


In [ ]:
threshold = 0.0
mask_list = []
while(True):
    threshold = threshold + 0.1
    rand_vect = np.random.rand(16) > threshold
    XE_intermediate = XE_intermediate_full[:,rand_vect]
    if threshold > 0.85:
        threshold = 0.0
    if XE_intermediate.shape[1] < 1:
        continue
        
    rand_vect_int = []
    for i in range(len(rand_vect)):
        if rand_vect[i] == True:
            rand_vect_int.append(1)
        else:
            rand_vect_int.append(0)
    if rand_vect_int in mask_list:
        continue

    limitE = int(0.8*len(XE_intermediate))
    #print(limitE)

    X_train = XE_intermediate[:limitE]
    X_test = XE_intermediate[limitE:]

    y_train = yE[:limitE]
    y_test = yE[limitE:]

    from sklearn.linear_model import SGDClassifier, LogisticRegression
    from sklearn.metrics import accuracy_score, f1_score

    logi = LogisticRegression()
    logi.fit(X_train, y_train)

    y_predicted = logi.predict(X_test)
    acc = accuracy_score(y_predicted, y_test)
    f1 = f1_score(y_predicted, y_test, average='weighted')

    y_predicted_proba = logi.predict_proba(X_test)
    y_predicted = (y_predicted_proba[:,1] > np.percentile(y_predicted_proba[:,1],100-28.5))

    acc = accuracy_score(y_predicted, y_test)
    f1 = f1_score(y_predicted, y_test, average='weighted')
    #print(acc, f1, sum(y_predicted), len(y_predicted))
    #print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
    #print("\n")
    
    rej = len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))
    acc = len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)
    
    X_test_df = pd.DataFrame(X_test)
    if (rej >= max_rej and acc >= max_acc):
        max_rej = rej
        
        max_acc = acc
        mask_list.append(rand_vect_int)
        if len(mask_list) > 2000:
            break
        print(rand_vect_int)
        print(len(mask_list))
        print("Rejected %.2f%% of wrong ones" % rej)
        print("Accepted %.2f%% of good ones" % acc)
        print("Median: %.3f\n\n" % np.median(y_predicted_proba[:,1]))


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1
Rejected 73.21% of wrong ones
Accepted 42.86% of good ones
Median: 0.149


[0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
2
Rejected 75.00% of wrong ones
Accepted 57.14% of good ones
Median: 0.151


[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]
3
Rejected 75.00% of wrong ones
Accepted 57.14% of good ones
Median: 0.165


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
4
Rejected 75.89% of wrong ones
Accepted 64.29% of good ones
Median: 0.166




/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1]
5
Rejected 75.89% of wrong ones
Accepted 64.29% of good ones
Median: 0.168


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
6
Rejected 77.68% of wrong ones
Accepted 78.57% of good ones
Median: 0.169


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
7
Rejected 77.68% of wrong ones
Accepted 78.57% of good ones
Median: 0.169


[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
8
Rejected 77.68% of wrong ones
Accepted 78.57% of good ones
Median: 0.169


[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
9
Rejected 77.68% of wrong ones
Accepted 78.57% of good ones
Median: 0.169


[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
10
Rejected 77.68% of wrong ones
Accepted 78.57% of good ones
Median: 0.169


[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
11
Rejected 77.68% of wrong ones
Accepted 78.57% of good ones
Median: 0.169


[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
12
Rejected 77.68% of wrong ones
Accepted 78.57% of good ones
Media

[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]
41
Rejected 72.32% of wrong ones
Accepted 78.57% of good ones
Median: 0.169


[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1]
42
Rejected 72.32% of wrong ones
Accepted 78.57% of good ones
Median: 0.169

percentile 33%

In [ ]:
mask_list_np = np.array([np.array(xi) for xi in mask_list])

In [ ]:
mask_list_np.sum(axis=0)/len(mask_list)

In [ ]:
a = mask_list_np.sum(axis=1)

import matplotlib.pyplot as plt
plt.hist(a, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram")
plt.show()

# Final testing

In [ ]:
rand_vect = mask_list[-1]
XE_intermediate = XE_intermediate_full[:,rand_vect]

In [104]:
limitE = int(0.8*len(XE_intermediate))
print(limitE)

X_train = XE_intermediate[:limitE]
X_test = XE_intermediate[limitE:]

y_train = yE[:limitE]
y_test = yE[limitE:]

500


In [111]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

logi = LogisticRegression()
logi.fit(X_train, y_train)

y_predicted = logi.predict(X_test)
acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')

y_predicted_proba = logi.predict_proba(X_test)
y_predicted = (y_predicted_proba[:,1] > np.percentile(y_predicted_proba[:,1],100-33))

acc = accuracy_score(y_predicted, y_test)
f1 = f1_score(y_predicted, y_test, average='weighted')
print(acc, f1, sum(y_predicted), len(y_predicted))
print(sum(y_predicted)*100.0/len(y_predicted),"% - ", int(sum(y_predicted)*50.0/len(y_predicted)))
print("\n")
X_test_df = pd.DataFrame(X_test)
print("Rejected %.2f%% of wrong ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == False)])*100.0/(len(y_test)-sum(y_test))))
print("Accepted %.2f%% of good ones" % (len(X_test_df[(y_predicted == y_test) & (y_predicted == True)])*100.0/sum(y_test)))
print("Median: %.3f" % np.median(y_predicted_proba[:,1]))

y_predicted_proba[0:5]

0.714285714286 0.670867756136 32 126
25.3968253968 % -  12


Rejected 75.89% of wrong ones
Accepted 35.71% of good ones
Median: 0.134


array([[ 0.9003563 ,  0.0996437 ],
       [ 0.87714897,  0.12285103],
       [ 0.86583697,  0.13416303],
       [ 0.87173636,  0.12826364],
       [ 0.75371774,  0.24628226]])